In [7]:
from el_toolkit.document import Document
from el_toolkit.lkb.wordnet_lkb import WordNet_Lexical_Knowledge_Base
from transformers import BertTokenizer


lkb = WordNet_Lexical_Knowledge_Base.read_json("data/knowledge_datasets/mesh_subset.json")

biomed_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='monologg/biobert_v1.1_pubmed',
                                            do_lower_case=False, cache_dir=None)
                                            
doc = Document.read_json("/Users/coltonflowers/Work/TREND_repos/BioMedical-EL/tests/test_data/test_docs/without_overlaps/14_mentions.json")
doc.jupyter_display(lkb)
for doc in doc.segment(biomed_tokenizer,8):
  doc.jupyter_display(lkb)
  print("-----------------------------------")

Mentions Omitted: 0


-----------------------------------


-----------------------------------


In [ ]:
from el_toolkit.lkb.wordnet_lkb import WordNet_Lexical_Knowledge_Base

In [ ]:
wn = WordNet_Lexical_Knowledge_Base.read_json("data/knowledge_datasets/trend.json")

In [ ]:
occupations = wn.derive_domain("C0028811","isa")

In [ ]:
len(occupations)

6137

In [4]:
from el_toolkit.models.dual_encoder.featurizer import  Featurizer


Featurizer


el_toolkit.models.dual_encoder.featurizer.Featurizer